## 데이터 로드 및 분리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv("/content/drive/MyDrive/boost/train.csv")
test = pd.read_csv("/content/drive/MyDrive/boost/test.csv")

In [4]:
# 0 : IT/Science
# 1 : Economy
# 2 : Society
# 3 : Culture
# 4 : World
# 5 : Sport
# 6 : Politics
it_science = train[train['target']==0]
economy = train[train['target']==1]
society = train[train['target']==2]
culture = train[train['target']==3]
world = train[train['target']==4]
sport = train[train['target']==5]
politics = train[train['target']==6]

## 형태소 분석 및 워드 카운트

In [5]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 41.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813256 sha256=6ea2218d484988ebde173a317bfd26bdb0f3ff5400739172fdb089c153103a94
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [6]:
# 사용자 함수 로드 후, 키위 형태소 분석 사용
from kiwipiepy import Kiwi
#from kiwipiepy.utils import Stopwords
kiwi = Kiwi()
# stopwords = Stopwords()
# kiwi.load_user_dictionary('')
# kiwi.prepare()

In [272]:
from tqdm import tqdm
tqdm.pandas()

1. 품사태깅

In [8]:
def pos_tagging(text):
  raw_pos_tagged = kiwi.tokenize(text, normalize_coda=False)
  return raw_pos_tagged

2. 품사 필터

In [79]:
## NN* : 명사 (일반, 고유, 의존)
## V* : 용언 (동사, 형용사)
## XR : 어근
## S* : 부호, 외국어, 특수문자 (알파벳, 숫자, 기타특수문자) # 'SL', 'SN', 'SW'
main = ['NNG', 'NNB', 'NNP', 'VV', 'VA', 'XR']
da = ['VV', 'VA']

def pos_filter(df, col):
  word_cleaned=[]
  for index, row in tqdm(df.iterrows()):
    if row[col]:
      tag_filter=[]
      for token in row[col]:
        if token.tag in main:
          if token.tag in da:
            tag_filter.append(token.form+'다')
          else:
            tag_filter.append(token.form)
    temp=[]
    for i in tag_filter:
      if len(i)>1:
        temp.append(i)
    word_cleaned.append(temp)
  return word_cleaned

3. 단어 카운트

In [80]:
def word_count(word_list):
  from collections import Counter
  result = Counter(word_list)
  word_dic = dict(result)
  sorted_dict = sorted(word_dic.items(), key= lambda item:item[1], reverse=True)
  return sorted_dict

### 0) it_science

In [177]:
it_science['text_pos_tagging'] = it_science['text'].progress_apply(lambda x: pos_tagging(x))

100%|██████████| 1000/1000 [00:01<00:00, 922.62it/s]
<ipython-input-177-180bbc7435d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  it_science['text_pos_tagging'] = it_science['text'].progress_apply(lambda x: pos_tagging(x))


In [178]:
it_science

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
1,ynat-v1_train_00001,삼성전자 KBIS 2018서 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33,"[(삼성전자, NNP, 0, 4), (KBIS, SL, 5, 4), (2018, S...","[삼성전자, 셰프, 컬렉션, 선보이다]"
2,ynat-v1_train_00002,LG G6 사면 BO 이어폰이 단돈 5천원,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.30. 오전 10:00,"[(LG, SL, 0, 2), (G, SL, 3, 1), (6, SN, 4, 1),...","[사면, 이어폰, 단돈]"
13,ynat-v1_train_00013,SK텔레콤 해킹불가능 장거리 양자암호통신 성공,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.06.19. 오전 9:18,"[(SK텔레콤, NNP, 0, 5), (해킹, NNG, 6, 2), (불, XPN,...","[SK텔레콤, 해킹, 가능, 장거리, 양자암호, 통신, 성공]"
17,ynat-v1_train_00017,마이크로소프트 2세대 무선 디스플레이 어댑터 출시,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.16. 오전 11:12,"[(마이크로소프트, NNP, 0, 7), (2, SN, 8, 1), (세대, NNG...","[마이크로소프트, 세대, 무선, 디스플레이, 어댑터, 출시]"
30,ynat-v1_train_00030,갤럭시S8 중동 상륙…28일 아랍에미리트 출시,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.12. 오전 9:24,"[(갤럭시S8, NNP, 0, 5), (중동, NNP, 6, 2), (상륙, NNG...","[갤럭시S8, 중동, 상륙, 아랍에미리트, 출시]"
...,...,...,...,...,...,...,...
6964,ynat-v1_train_06964,구글 국립공원 탐방로 파노라마 이미지 제공,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.06.22. 오후 3:52,"[(구글, NNP, 0, 2), (국립공원, NNG, 3, 4), (탐방로, NNG...","[구글, 국립공원, 탐방로, 파노라마, 이미지, 제공]"
6967,ynat-v1_train_06967,2018 ICT 결산 ③플랫폼 업계 위기 넘으면 또 다른 위기끝,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.12.16. 오전 6:03,"[(2018, SN, 0, 4), (ICT, SL, 5, 3), (결산, NNG, ...","[결산, 플랫폼, 업계, 위기, 넘다, 위기]"
6977,ynat-v1_train_06977,KT 28㎓ 대역에서 5G 네트워크 현장 검증 성공,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.01.15. 오전 10:40,"[(KT, SL, 0, 2), (28, SN, 3, 2), (㎓, SW, 5, 1)...","[네트워크, 현장, 검증, 성공]"
6980,ynat-v1_train_06980,태양열로 공기 중 물 얻는다…물 부족 국가에 유용,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.14. 오전 3:00,"[(태양열, NNG, 0, 3), (로, JKB, 3, 1), (공기, NNG, 5...","[태양열, 공기, 부족, 국가, 유용]"


In [179]:
it_science['text_pos_filter'] = pos_filter(it_science,'text_pos_tagging')

1000it [00:00, 16274.53it/s]
<ipython-input-179-5918fc3834e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  it_science['text_pos_filter'] = pos_filter(it_science,'text_pos_tagging')


In [180]:
it_science

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
1,ynat-v1_train_00001,삼성전자 KBIS 2018서 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33,"[(삼성전자, NNP, 0, 4), (KBIS, SL, 5, 4), (2018, S...","[삼성전자, 셰프, 컬렉션, 선보이다]"
2,ynat-v1_train_00002,LG G6 사면 BO 이어폰이 단돈 5천원,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.30. 오전 10:00,"[(LG, SL, 0, 2), (G, SL, 3, 1), (6, SN, 4, 1),...","[사면, 이어폰, 단돈]"
13,ynat-v1_train_00013,SK텔레콤 해킹불가능 장거리 양자암호통신 성공,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.06.19. 오전 9:18,"[(SK텔레콤, NNP, 0, 5), (해킹, NNG, 6, 2), (불, XPN,...","[SK텔레콤, 해킹, 가능, 장거리, 양자암호, 통신, 성공]"
17,ynat-v1_train_00017,마이크로소프트 2세대 무선 디스플레이 어댑터 출시,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.16. 오전 11:12,"[(마이크로소프트, NNP, 0, 7), (2, SN, 8, 1), (세대, NNG...","[마이크로소프트, 세대, 무선, 디스플레이, 어댑터, 출시]"
30,ynat-v1_train_00030,갤럭시S8 중동 상륙…28일 아랍에미리트 출시,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.12. 오전 9:24,"[(갤럭시S8, NNP, 0, 5), (중동, NNP, 6, 2), (상륙, NNG...","[갤럭시S8, 중동, 상륙, 아랍에미리트, 출시]"
...,...,...,...,...,...,...,...
6964,ynat-v1_train_06964,구글 국립공원 탐방로 파노라마 이미지 제공,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.06.22. 오후 3:52,"[(구글, NNP, 0, 2), (국립공원, NNG, 3, 4), (탐방로, NNG...","[구글, 국립공원, 탐방로, 파노라마, 이미지, 제공]"
6967,ynat-v1_train_06967,2018 ICT 결산 ③플랫폼 업계 위기 넘으면 또 다른 위기끝,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.12.16. 오전 6:03,"[(2018, SN, 0, 4), (ICT, SL, 5, 3), (결산, NNG, ...","[결산, 플랫폼, 업계, 위기, 넘다, 위기]"
6977,ynat-v1_train_06977,KT 28㎓ 대역에서 5G 네트워크 현장 검증 성공,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.01.15. 오전 10:40,"[(KT, SL, 0, 2), (28, SN, 3, 2), (㎓, SW, 5, 1)...","[네트워크, 현장, 검증, 성공]"
6980,ynat-v1_train_06980,태양열로 공기 중 물 얻는다…물 부족 국가에 유용,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.14. 오전 3:00,"[(태양열, NNG, 0, 3), (로, JKB, 3, 1), (공기, NNG, 5...","[태양열, 공기, 부족, 국가, 유용]"


In [181]:
it_science_cnt = list(it_science.loc[:, 'text_pos_filter'])
it_science_cnt = [data for inner_list in it_science_cnt for data in inner_list]
it_science_cnt = word_count(it_science_cnt)
it_science_cnt[:30]

[('출시', 100),
 ('개발', 80),
 ('기술', 74),
 ('종합', 69),
 ('삼성', 47),
 ('스마트폰', 38),
 ('네이버', 35),
 ('공개', 35),
 ('국내', 33),
 ('게시판', 32),
 ('서비스', 31),
 ('데이터', 27),
 ('한국', 26),
 ('구글', 25),
 ('돌파', 25),
 ('연구', 25),
 ('판매', 24),
 ('인터넷', 24),
 ('갤럭시', 24),
 ('영상', 23),
 ('스마트', 23),
 ('카카오', 23),
 ('아이폰', 23),
 ('통신', 22),
 ('상용', 22),
 ('가능', 21),
 ('세계', 21),
 ('애플', 21),
 ('과학', 20),
 ('기능', 20)]

In [182]:
len(it_science_cnt)

2713

In [183]:
it_science_df = pd.DataFrame(it_science_cnt,columns=['word','cnt'])
it_science_df

,word,cnt
0,출시,100
1,개발,80
2,기술,74
3,종합,69
4,삼성,47
...,...,...
2708,이미지,1
2709,공기,1
2710,슬로우,1
2711,슬립,1


In [184]:
it_science_drop_df = it_science_df[it_science_df['cnt']>9]
it_science_drop_df

,word,cnt
0,출시,100
1,개발,80
2,기술,74
3,종합,69
4,삼성,47
...,...,...
84,공동,10
85,확인,10
86,알파고,10
87,활용,10


In [185]:
it_science_df.to_csv('it_science_df.csv',encoding='utf-8-sig')

In [186]:
it_science_drop_df.to_csv('it_science_drop_df.csv',encoding='utf-8-sig')

### 1) economy


In [187]:
economy['text_pos_tagging'] = economy['text'].progress_apply(lambda x: pos_tagging(x))

100%|██████████| 1000/1000 [00:01<00:00, 935.74it/s]
<ipython-input-187-d3cb0eeec80e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  economy['text_pos_tagging'] = economy['text'].progress_apply(lambda x: pos_tagging(x))


In [188]:
economy

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
0,ynat-v1_train_00000,개포2단지 분양 앞두고 개포지구 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37,"[(개포, NNP, 0, 2), (2, SN, 2, 1), (단지, NNG, 3, ...","[개포, 단지, 분양, 앞두다, 개포, 지구, 건축, 불붙다]"
6,ynat-v1_train_00006,콜마비앤에이치 작년 영업익 352억원…54.5%↑,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.01.18. 오후 4:45,"[(콜, NNG, 0, 1), (마비, NNG, 1, 2), (앤, NNG, 3, ...","[마비, 에이치, 작년, 영업익]"
7,ynat-v1_train_00007,SOil 1조2천억원 경유·납사·항공유 공급계약,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.05. 오전 10:21,"[(SOil, SL, 0, 4), (1, SN, 5, 1), (조, NR, 6, 1...","[경유, 납사, 항공유, 공급, 계약]"
8,ynat-v1_train_00008,카카오·삼성화재 디지털 손해보험사 설립 추진,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.23. 오전 10:49,"[(카카오, NNP, 0, 3), (·, SP, 3, 1), (삼성화재, NNP, ...","[카카오, 삼성화재, 디지털, 손해, 보험사, 설립, 추진]"
11,ynat-v1_train_00011,그래픽 편볍증여 통한 부동산 취득 사례,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.11.12. 오후 3:06,"[(그래픽, NNG, 0, 3), (편볍, NNG, 4, 2), (증여, NNG, ...","[그래픽, 편볍, 증여, 통하다, 부동산, 취득, 사례]"
...,...,...,...,...,...,...,...
6969,ynat-v1_train_06969,소녀상과 함께,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.11.22. 오후 5:02,"[(소녀상, NNG, 0, 3), (과, JKB, 3, 1), (함께, MAG, 5...",[소녀상]
6974,ynat-v1_train_06974,그래픽 주간 아파트값 변동률 추이,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.31. 오후 2:18,"[(그래픽, NNG, 0, 3), (주간, NNG, 4, 2), (아파트값, NNG...","[그래픽, 주간, 아파트값, 변동, 추이]"
6978,ynat-v1_train_06978,SK텔레콤 기기오처노배궈니면 스마트호 묘금 걱쩡 끋,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.10.24. 오전 10:36,"[(SK텔레콤, NNP, 0, 5), (기기오처노배궈니면, NNP, 6, 9), (...","[SK텔레콤, 기기오처노배궈니면, 스마트호, 묘금, 걱쩡]"
6987,ynat-v1_train_06987,게시판 유안타증권 해외주식 비과세펀드 출시 이벤트,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.02.29. 오전 10:15,"[(게시판, NNG, 0, 3), (유안타증권, NNP, 4, 5), (해외, NN...","[게시판, 유안타증권, 해외, 주식, 과세, 펀드, 출시, 이벤트]"


In [189]:
economy['text_pos_filter'] = pos_filter(economy,'text_pos_tagging')

1000it [00:00, 15336.56it/s]
<ipython-input-189-ecdbed593eca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  economy['text_pos_filter'] = pos_filter(economy,'text_pos_tagging')


In [190]:
economy

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
0,ynat-v1_train_00000,개포2단지 분양 앞두고 개포지구 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37,"[(개포, NNP, 0, 2), (2, SN, 2, 1), (단지, NNG, 3, ...","[개포, 단지, 분양, 앞두다, 개포, 지구, 건축, 불붙다]"
6,ynat-v1_train_00006,콜마비앤에이치 작년 영업익 352억원…54.5%↑,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.01.18. 오후 4:45,"[(콜, NNG, 0, 1), (마비, NNG, 1, 2), (앤, NNG, 3, ...","[마비, 에이치, 작년, 영업익]"
7,ynat-v1_train_00007,SOil 1조2천억원 경유·납사·항공유 공급계약,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.05. 오전 10:21,"[(SOil, SL, 0, 4), (1, SN, 5, 1), (조, NR, 6, 1...","[경유, 납사, 항공유, 공급, 계약]"
8,ynat-v1_train_00008,카카오·삼성화재 디지털 손해보험사 설립 추진,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.23. 오전 10:49,"[(카카오, NNP, 0, 3), (·, SP, 3, 1), (삼성화재, NNP, ...","[카카오, 삼성화재, 디지털, 손해, 보험사, 설립, 추진]"
11,ynat-v1_train_00011,그래픽 편볍증여 통한 부동산 취득 사례,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.11.12. 오후 3:06,"[(그래픽, NNG, 0, 3), (편볍, NNG, 4, 2), (증여, NNG, ...","[그래픽, 편볍, 증여, 통하다, 부동산, 취득, 사례]"
...,...,...,...,...,...,...,...
6969,ynat-v1_train_06969,소녀상과 함께,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.11.22. 오후 5:02,"[(소녀상, NNG, 0, 3), (과, JKB, 3, 1), (함께, MAG, 5...",[소녀상]
6974,ynat-v1_train_06974,그래픽 주간 아파트값 변동률 추이,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.31. 오후 2:18,"[(그래픽, NNG, 0, 3), (주간, NNG, 4, 2), (아파트값, NNG...","[그래픽, 주간, 아파트값, 변동, 추이]"
6978,ynat-v1_train_06978,SK텔레콤 기기오처노배궈니면 스마트호 묘금 걱쩡 끋,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.10.24. 오전 10:36,"[(SK텔레콤, NNP, 0, 5), (기기오처노배궈니면, NNP, 6, 9), (...","[SK텔레콤, 기기오처노배궈니면, 스마트호, 묘금, 걱쩡]"
6987,ynat-v1_train_06987,게시판 유안타증권 해외주식 비과세펀드 출시 이벤트,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.02.29. 오전 10:15,"[(게시판, NNG, 0, 3), (유안타증권, NNP, 4, 5), (해외, NN...","[게시판, 유안타증권, 해외, 주식, 과세, 펀드, 출시, 이벤트]"


In [191]:
economy_cnt = list(economy.loc[:, 'text_pos_filter'])
economy_cnt = [data for inner_list in economy_cnt for data in inner_list]
economy_cnt = word_count(economy_cnt)
economy_cnt[:30]

[('종합', 92),
 ('분기', 91),
 ('투자', 71),
 ('작년', 48),
 ('실적', 44),
 ('시장', 44),
 ('금융', 42),
 ('출시', 37),
 ('영업익', 36),
 ('특징', 35),
 ('증가', 35),
 ('금리', 34),
 ('올해', 33),
 ('게시판', 33),
 ('코스피', 32),
 ('대출', 31),
 ('상승', 30),
 ('증권', 30),
 ('주식', 29),
 ('달러', 29),
 ('목표', 29),
 ('기업', 28),
 ('하락', 26),
 ('삼성전자', 24),
 ('상장', 24),
 ('영업', 24),
 ('은행', 24),
 ('펀드', 23),
 ('규모', 23),
 ('이익', 23)]

In [192]:
len(economy_cnt)

2681

In [193]:
economy_df = pd.DataFrame(economy_cnt,columns=['word','cnt'])
economy_df

,word,cnt
0,종합,92
1,분기,91
2,투자,71
3,작년,48
4,실적,44
...,...,...
2676,소녀상,1
2677,기기오처노배궈니면,1
2678,스마트호,1
2679,묘금,1


In [194]:
economy_drop_df = economy_df[economy_df['cnt']>9]
economy_drop_df

,word,cnt
0,종합,92
1,분기,91
2,투자,71
3,작년,48
4,실적,44
...,...,...
101,개월,10
102,추가,10
103,개선,10
104,업체,10


In [195]:
economy_df.to_csv('economy_df.csv',encoding='utf-8-sig')

In [196]:
economy_drop_df.to_csv('economy_drop_df.csv',encoding='utf-8-sig')

### 2) society

In [197]:
society['text_pos_tagging'] = society['text'].progress_apply(lambda x: pos_tagging(x))

100%|██████████| 1000/1000 [00:01<00:00, 839.72it/s]
<ipython-input-197-f51b3eb34f2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  society['text_pos_tagging'] = society['text'].progress_apply(lambda x: pos_tagging(x))


In [198]:
society

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
18,ynat-v1_train_00018,조배숙 중앙위 정례회의 주재,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.02.02. 오전 8:54,"[(조배숙, NNP, 0, 3), (중앙위, NNG, 4, 3), (정례, NNG,...","[조배숙, 중앙위, 정례, 회의, 주재]"
25,ynat-v1_train_00025,과기정통부 올해 이동통신 데이터요금 18% 인하할 것,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.24. 오전 9:00,"[(과기, NNP, 0, 2), (정통부, NNG, 2, 3), (올해, NNG, ...","[과기, 정통부, 올해, 이동, 통신, 데이터, 요금, 인하]"
28,ynat-v1_train_00028,파리바게뜨 갈등 봉합됐지만…긍정적 발판 vs 부정적 선례,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.14. 오전 6:07,"[(파리바게뜨, NNP, 0, 5), (갈등, NNG, 6, 2), (봉합, NNG...","[파리바게뜨, 갈등, 봉합, 긍정, 발판, 부정, 선례]"
35,ynat-v1_train_00035,與 대구·경북 공천면접심사…현역 vs 진박 격돌,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.02.26. 오전 5:00,"[(與, SH, 0, 1), (대구, NNP, 2, 2), (·, SP, 4, 1)...","[대구, 경북, 공천, 면접, 심사, 현역, 진박, 격돌]"
41,ynat-v1_train_00041,증명사진 없어도 재외공관서 전자여권 신청 가능,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.12.30. 오전 11:27,"[(증명사진, NNG, 0, 4), (없, VA, 5, 1), (어도, EC, 6,...","[증명사진, 없다, 재외, 공관, 전자, 여권, 신청, 가능]"
...,...,...,...,...,...,...,...
6955,ynat-v1_train_06955,1608년 지도가 말하는 사실 남중국해는 상인의 바다,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.11.02. 오후 5:04,"[(1608, SN, 0, 4), (년, NNB, 4, 1), (지도, NNG, 6...","[지도, 남중국해, 상인, 바다]"
6960,ynat-v1_train_06960,해양경찰 새 옷을 입고 국민에게 다가갑니다,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.04.19. 오전 9:14,"[(해양, NNG, 0, 2), (경찰, NNG, 2, 2), (새, MM, 5, ...","[해양, 경찰, 국민, 다가가다]"
6965,ynat-v1_train_06965,발표하는 홍민정 사교육걱정없는세상 대표,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.03.10. 오후 2:38,"[(발표, NNG, 0, 2), (하, XSV, 2, 1), (는, ETM, 3, ...","[발표, 민정, 사교육, 걱정, 없다, 세상, 대표]"
6973,ynat-v1_train_06973,언론노조 창립 30주년 기념식,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.11.23. 오후 7:57,"[(언론, NNG, 0, 2), (노조, NNG, 2, 2), (창립, NNG, 5...","[언론, 노조, 창립, 주년, 기념식]"


In [199]:
society['text_pos_filter'] = pos_filter(society,'text_pos_tagging')

1000it [00:00, 13662.94it/s]
<ipython-input-199-b51817ef44b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  society['text_pos_filter'] = pos_filter(society,'text_pos_tagging')


In [200]:
society

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
18,ynat-v1_train_00018,조배숙 중앙위 정례회의 주재,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.02.02. 오전 8:54,"[(조배숙, NNP, 0, 3), (중앙위, NNG, 4, 3), (정례, NNG,...","[조배숙, 중앙위, 정례, 회의, 주재]"
25,ynat-v1_train_00025,과기정통부 올해 이동통신 데이터요금 18% 인하할 것,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.24. 오전 9:00,"[(과기, NNP, 0, 2), (정통부, NNG, 2, 3), (올해, NNG, ...","[과기, 정통부, 올해, 이동, 통신, 데이터, 요금, 인하]"
28,ynat-v1_train_00028,파리바게뜨 갈등 봉합됐지만…긍정적 발판 vs 부정적 선례,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.14. 오전 6:07,"[(파리바게뜨, NNP, 0, 5), (갈등, NNG, 6, 2), (봉합, NNG...","[파리바게뜨, 갈등, 봉합, 긍정, 발판, 부정, 선례]"
35,ynat-v1_train_00035,與 대구·경북 공천면접심사…현역 vs 진박 격돌,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.02.26. 오전 5:00,"[(與, SH, 0, 1), (대구, NNP, 2, 2), (·, SP, 4, 1)...","[대구, 경북, 공천, 면접, 심사, 현역, 진박, 격돌]"
41,ynat-v1_train_00041,증명사진 없어도 재외공관서 전자여권 신청 가능,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.12.30. 오전 11:27,"[(증명사진, NNG, 0, 4), (없, VA, 5, 1), (어도, EC, 6,...","[증명사진, 없다, 재외, 공관, 전자, 여권, 신청, 가능]"
...,...,...,...,...,...,...,...
6955,ynat-v1_train_06955,1608년 지도가 말하는 사실 남중국해는 상인의 바다,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.11.02. 오후 5:04,"[(1608, SN, 0, 4), (년, NNB, 4, 1), (지도, NNG, 6...","[지도, 남중국해, 상인, 바다]"
6960,ynat-v1_train_06960,해양경찰 새 옷을 입고 국민에게 다가갑니다,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.04.19. 오전 9:14,"[(해양, NNG, 0, 2), (경찰, NNG, 2, 2), (새, MM, 5, ...","[해양, 경찰, 국민, 다가가다]"
6965,ynat-v1_train_06965,발표하는 홍민정 사교육걱정없는세상 대표,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.03.10. 오후 2:38,"[(발표, NNG, 0, 2), (하, XSV, 2, 1), (는, ETM, 3, ...","[발표, 민정, 사교육, 걱정, 없다, 세상, 대표]"
6973,ynat-v1_train_06973,언론노조 창립 30주년 기념식,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.11.23. 오후 7:57,"[(언론, NNG, 0, 2), (노조, NNG, 2, 2), (창립, NNG, 5...","[언론, 노조, 창립, 주년, 기념식]"


In [201]:
society_cnt = list(society.loc[:, 'text_pos_filter'])
society_cnt = [data for inner_list in society_cnt for data in inner_list]
society_cnt = word_count(society_cnt)
society_cnt[:30]

[('종합', 77),
 ('게시판', 43),
 ('코로나', 40),
 ('지원', 29),
 ('한국', 24),
 ('교육', 23),
 ('노조', 23),
 ('대표', 21),
 ('학생', 21),
 ('광주', 20),
 ('사장', 20),
 ('방송', 19),
 ('부산', 18),
 ('학교', 17),
 ('하다', 17),
 ('기자', 16),
 ('안전', 16),
 ('공공', 16),
 ('일자리', 16),
 ('노동자', 16),
 ('뉴스', 15),
 ('검찰', 15),
 ('이사', 15),
 ('대전', 15),
 ('공무원', 15),
 ('통신', 14),
 ('강원', 14),
 ('대학', 14),
 ('강화', 14),
 ('기업', 14)]

In [202]:
len(society_cnt)

3473

In [203]:
society_df = pd.DataFrame(society_cnt,columns=['word','cnt'])
society_df

,word,cnt
0,종합,77
1,게시판,43
2,코로나,40
3,지원,29
4,한국,24
...,...,...
3468,걱정,1
3469,세상,1
3470,업또록,1
3471,쪼용히,1


In [204]:
society_drop_df = society_df[society_df['cnt']>9]
society_drop_df

,word,cnt
0,종합,77
1,게시판,43
2,코로나,40
3,지원,29
4,한국,24
...,...,...
78,등교,10
79,정책,10
80,시민,10
81,간담회,10


In [205]:
society_df.to_csv('society_df.csv',encoding='utf-8-sig')

In [206]:
society_drop_df.to_csv('society_drop_df.csv',encoding='utf-8-sig')

### 3) culture

In [207]:
culture['text_pos_tagging'] = culture['text'].progress_apply(lambda x: pos_tagging(x))

100%|██████████| 1000/1000 [00:01<00:00, 878.67it/s]
<ipython-input-207-75790eefad5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  culture['text_pos_tagging'] = culture['text'].progress_apply(lambda x: pos_tagging(x))


In [208]:
culture

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
3,ynat-v1_train_00003,신간 블록체인혁명 2030·남자의 고독사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.13. 오전 11:49,"[(신간, NNG, 0, 2), (블록체인, NNP, 3, 4), (혁명, NNG,...","[신간, 블록체인, 혁명, 남자, 고독사]"
10,ynat-v1_train_00010,폭염 이기는 도심 속 오페라 공연,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.07.21. 오전 10:15,"[(폭염, NNG, 0, 2), (이기, VV, 3, 2), (는, ETM, 5, ...","[폭염, 이기다, 도심, 오페라, 공연]"
19,ynat-v1_train_00019,경북·여수·정선·창원 관광발전지수 1등급 지역에 새로 선정,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.30. 오전 11:04,"[(경북, NNP, 0, 2), (·, SP, 2, 1), (여수, NNP, 3, ...","[경북, 여수, 정선, 창원, 관광, 발전, 지수, 등급, 지역, 선정]"
23,ynat-v1_train_00023,그래픽 부산해남 최장 걷기여행길 남파랑길,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.03.25. 오후 4:37,"[(그래픽, NNG, 0, 3), (부산, NNP, 4, 2), (해남, NNP, ...","[그래픽, 부산, 해남, 최장, 걷기, 여행, 남파]"
26,ynat-v1_train_00026,책으로 떠나는 한양도성 순례…성곽길시간여행 출간,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.12.11. 오전 7:30,"[(책, NNG, 0, 1), (으로, JKB, 1, 2), (떠나, VV, 4, ...","[떠나다, 한양도성, 순례, 성곽, 시간, 여행, 출간]"
...,...,...,...,...,...,...,...
6966,ynat-v1_train_06966,신간 식민지 근대감옥 서대문형무소,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.04.12. 오후 3:25,"[(신간, NNG, 0, 2), (식민지, NNG, 3, 3), (근대, NNG, ...","[신간, 식민지, 근대, 감옥, 서대문형무소]"
6981,ynat-v1_train_06981,미세먼지 싫어요,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.17. 오후 1:44,"[(미세, NNG, 0, 2), (먼지, NNG, 2, 2), (싫, VA, 5, ...","[미세, 먼지, 싫다]"
6983,ynat-v1_train_06983,신간 무고한 존재·드롭·유령함대,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.03.15. 오후 4:00,"[(신간, NNG, 0, 2), (무고, XR, 3, 2), (하, XSA, 5, ...","[신간, 무고, 존재, 드롭, 유령, 함대]"
6991,ynat-v1_train_06991,서울시향 슈텐츠 지휘의 정기연주회 성료,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.06.24. 오후 4:13,"[(서울시향, NNP, 0, 4), (슈텐츠, NNP, 5, 3), (지휘, NNG...","[서울시향, 슈텐츠, 지휘, 정기, 연주회]"


In [209]:
culture['text_pos_filter'] = pos_filter(culture,'text_pos_tagging')

1000it [00:00, 11830.70it/s]
<ipython-input-209-7c59c5097b59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  culture['text_pos_filter'] = pos_filter(culture,'text_pos_tagging')


In [210]:
culture

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
3,ynat-v1_train_00003,신간 블록체인혁명 2030·남자의 고독사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.13. 오전 11:49,"[(신간, NNG, 0, 2), (블록체인, NNP, 3, 4), (혁명, NNG,...","[신간, 블록체인, 혁명, 남자, 고독사]"
10,ynat-v1_train_00010,폭염 이기는 도심 속 오페라 공연,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.07.21. 오전 10:15,"[(폭염, NNG, 0, 2), (이기, VV, 3, 2), (는, ETM, 5, ...","[폭염, 이기다, 도심, 오페라, 공연]"
19,ynat-v1_train_00019,경북·여수·정선·창원 관광발전지수 1등급 지역에 새로 선정,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.30. 오전 11:04,"[(경북, NNP, 0, 2), (·, SP, 2, 1), (여수, NNP, 3, ...","[경북, 여수, 정선, 창원, 관광, 발전, 지수, 등급, 지역, 선정]"
23,ynat-v1_train_00023,그래픽 부산해남 최장 걷기여행길 남파랑길,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.03.25. 오후 4:37,"[(그래픽, NNG, 0, 3), (부산, NNP, 4, 2), (해남, NNP, ...","[그래픽, 부산, 해남, 최장, 걷기, 여행, 남파]"
26,ynat-v1_train_00026,책으로 떠나는 한양도성 순례…성곽길시간여행 출간,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.12.11. 오전 7:30,"[(책, NNG, 0, 1), (으로, JKB, 1, 2), (떠나, VV, 4, ...","[떠나다, 한양도성, 순례, 성곽, 시간, 여행, 출간]"
...,...,...,...,...,...,...,...
6966,ynat-v1_train_06966,신간 식민지 근대감옥 서대문형무소,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.04.12. 오후 3:25,"[(신간, NNG, 0, 2), (식민지, NNG, 3, 3), (근대, NNG, ...","[신간, 식민지, 근대, 감옥, 서대문형무소]"
6981,ynat-v1_train_06981,미세먼지 싫어요,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.17. 오후 1:44,"[(미세, NNG, 0, 2), (먼지, NNG, 2, 2), (싫, VA, 5, ...","[미세, 먼지, 싫다]"
6983,ynat-v1_train_06983,신간 무고한 존재·드롭·유령함대,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.03.15. 오후 4:00,"[(신간, NNG, 0, 2), (무고, XR, 3, 2), (하, XSA, 5, ...","[신간, 무고, 존재, 드롭, 유령, 함대]"
6991,ynat-v1_train_06991,서울시향 슈텐츠 지휘의 정기연주회 성료,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.06.24. 오후 4:13,"[(서울시향, NNP, 0, 4), (슈텐츠, NNP, 5, 3), (지휘, NNG...","[서울시향, 슈텐츠, 지휘, 정기, 연주회]"


In [211]:
culture_cnt = list(culture.loc[:, 'text_pos_filter'])
culture_cnt = [data for inner_list in culture_cnt for data in inner_list]
culture_cnt = word_count(culture_cnt)
culture_cnt[:30]

[('신간', 103),
 ('여행', 48),
 ('축제', 44),
 ('서울', 39),
 ('공연', 38),
 ('주말', 36),
 ('전국', 33),
 ('날씨', 33),
 ('문화', 32),
 ('주의보', 28),
 ('한국', 28),
 ('개막', 27),
 ('개최', 24),
 ('강원', 24),
 ('종합', 21),
 ('제주', 21),
 ('출간', 20),
 ('한파', 20),
 ('소설', 20),
 ('기온', 20),
 ('가을', 19),
 ('게시판', 19),
 ('작가', 19),
 ('보다', 19),
 ('최고', 18),
 ('부산', 17),
 ('미세', 17),
 ('먼지', 17),
 ('관광', 16),
 ('쓰다', 16)]

In [212]:
len(culture_cnt)

2992

In [213]:
culture_df = pd.DataFrame(culture_cnt,columns=['word','cnt'])
culture_df

,word,cnt
0,신간,103
1,여행,48
2,축제,44
3,서울,39
4,공연,38
...,...,...
2987,서울시향,1
2988,슈텐츠,1
2989,정기,1
2990,대한제국,1


In [214]:
culture_drop_df = culture_df[culture_df['cnt']>9]
culture_drop_df

,word,cnt
0,신간,103
1,여행,48
2,축제,44
3,서울,39
4,공연,38
...,...,...
68,남부,10
69,서다,10
70,예술,10
71,국제,10


In [215]:
culture_df.to_csv('culture_df.csv',encoding='utf-8-sig')

In [216]:
culture_drop_df.to_csv('culture_drop_df.csv',encoding='utf-8-sig')

### 4) world

In [217]:
world['text_pos_tagging'] = world['text'].progress_apply(lambda x: pos_tagging(x))

100%|██████████| 1000/1000 [00:01<00:00, 843.02it/s]
<ipython-input-217-c747bc26886a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  world['text_pos_tagging'] = world['text'].progress_apply(lambda x: pos_tagging(x))


In [218]:
world

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
4,ynat-v1_train_00004,이스라엘 정보당국 팔레스타인인 50명 테러 혐의로 체포,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.18. 오후 11:15,"[(이스라엘, NNP, 0, 4), (정보, NNG, 5, 2), (당국, NNG,...","[이스라엘, 정보, 당국, 팔레스타인인, 테러, 혐의, 체포]"
12,ynat-v1_train_00012,美웨스트버지니아 경선 샌더스 박빙 우세…전체 판세와는 무관,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.11. 오전 9:48,"[(美, SH, 0, 1), (웨스트버지니아, NNP, 1, 7), (경선, NNG...","[웨스트버지니아, 경선, 샌더스, 박빙, 우세, 전체, 판세, 무관]"
54,ynat-v1_train_00054,이란 호르무즈 부근서 美드론 격추…美 이유없는 공격종합3보,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.20. 오후 9:46,"[(이란, NNP, 0, 2), (호르무즈, NNP, 3, 4), (부근, NNG,...","[이란, 호르무즈, 부근, 드론, 격추, 이유, 없다, 공격, 종합]"
56,ynat-v1_train_00056,테러 빈번한 부르키나파소…수년째 이슬람 무장단체 활개,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.05.13. 오후 6:07,"[(테러, NNG, 0, 2), (빈번, XR, 3, 2), (하, XSA, 5, ...","[테러, 빈번, 부르키나파소, 수년째, 이슬람, 무장, 단체, 활개]"
65,ynat-v1_train_00065,또다시 중동 벌집쑤신 트럼프…골란고원 군사긴장 고조,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.03.26. 오후 7:39,"[(또다시, MAG, 0, 3), (중동, NNP, 4, 2), (벌집, NNG, ...","[중동, 벌집, 쑤시다, 트럼프, 고르다, 고원, 군사, 긴장, 고조]"
...,...,...,...,...,...,...,...
6956,ynat-v1_train_06956,군사원조 지연 설명 못들어…우크라대통령 트럼프압력설 일축,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.02. 오전 10:05,"[(군사, NNG, 0, 2), (원조, NNG, 2, 2), (지연, NNG, 5...","[군사, 원조, 지연, 설명, 들다, 우크라, 대통령, 트럼프, 압력, 일축]"
6968,ynat-v1_train_06968,터키 러시아제 미사일 도입에 나토 최악의 위기,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.07.19. 오전 11:31,"[(터키, NNP, 0, 2), (러시아제, NNG, 3, 4), (미사일, NNG...","[터키, 러시아제, 미사일, 도입, 나토, 최악, 위기]"
6970,ynat-v1_train_06970,중국 무역전쟁 속 최대 이벤트 양회 개막종합2보,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.03.03. 오후 5:40,"[(중국, NNP, 0, 2), (무역, NNG, 3, 2), (전쟁, NNG, 5...","[중국, 무역, 전쟁, 최대, 이벤트, 양회, 개막, 종합]"
6972,ynat-v1_train_06972,伊 외무부 對쿠르드 군사작전에 항의…터키 대사 초치종합,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.11. 오전 12:48,"[(伊, SH, 0, 1), (외무부, NNG, 2, 3), (對, SH, 6, 1...","[외무부, 쿠르드, 군사, 작전, 항의, 터키, 대사, 초치, 종합]"


In [219]:
world['text_pos_filter'] = pos_filter(world,'text_pos_tagging')

1000it [00:00, 14814.37it/s]
<ipython-input-219-4e14ee9641e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  world['text_pos_filter'] = pos_filter(world,'text_pos_tagging')


In [220]:
world

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
4,ynat-v1_train_00004,이스라엘 정보당국 팔레스타인인 50명 테러 혐의로 체포,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.18. 오후 11:15,"[(이스라엘, NNP, 0, 4), (정보, NNG, 5, 2), (당국, NNG,...","[이스라엘, 정보, 당국, 팔레스타인인, 테러, 혐의, 체포]"
12,ynat-v1_train_00012,美웨스트버지니아 경선 샌더스 박빙 우세…전체 판세와는 무관,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.11. 오전 9:48,"[(美, SH, 0, 1), (웨스트버지니아, NNP, 1, 7), (경선, NNG...","[웨스트버지니아, 경선, 샌더스, 박빙, 우세, 전체, 판세, 무관]"
54,ynat-v1_train_00054,이란 호르무즈 부근서 美드론 격추…美 이유없는 공격종합3보,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.20. 오후 9:46,"[(이란, NNP, 0, 2), (호르무즈, NNP, 3, 4), (부근, NNG,...","[이란, 호르무즈, 부근, 드론, 격추, 이유, 없다, 공격, 종합]"
56,ynat-v1_train_00056,테러 빈번한 부르키나파소…수년째 이슬람 무장단체 활개,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.05.13. 오후 6:07,"[(테러, NNG, 0, 2), (빈번, XR, 3, 2), (하, XSA, 5, ...","[테러, 빈번, 부르키나파소, 수년째, 이슬람, 무장, 단체, 활개]"
65,ynat-v1_train_00065,또다시 중동 벌집쑤신 트럼프…골란고원 군사긴장 고조,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.03.26. 오후 7:39,"[(또다시, MAG, 0, 3), (중동, NNP, 4, 2), (벌집, NNG, ...","[중동, 벌집, 쑤시다, 트럼프, 고르다, 고원, 군사, 긴장, 고조]"
...,...,...,...,...,...,...,...
6956,ynat-v1_train_06956,군사원조 지연 설명 못들어…우크라대통령 트럼프압력설 일축,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.02. 오전 10:05,"[(군사, NNG, 0, 2), (원조, NNG, 2, 2), (지연, NNG, 5...","[군사, 원조, 지연, 설명, 들다, 우크라, 대통령, 트럼프, 압력, 일축]"
6968,ynat-v1_train_06968,터키 러시아제 미사일 도입에 나토 최악의 위기,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.07.19. 오전 11:31,"[(터키, NNP, 0, 2), (러시아제, NNG, 3, 4), (미사일, NNG...","[터키, 러시아제, 미사일, 도입, 나토, 최악, 위기]"
6970,ynat-v1_train_06970,중국 무역전쟁 속 최대 이벤트 양회 개막종합2보,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.03.03. 오후 5:40,"[(중국, NNP, 0, 2), (무역, NNG, 3, 2), (전쟁, NNG, 5...","[중국, 무역, 전쟁, 최대, 이벤트, 양회, 개막, 종합]"
6972,ynat-v1_train_06972,伊 외무부 對쿠르드 군사작전에 항의…터키 대사 초치종합,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.11. 오전 12:48,"[(伊, SH, 0, 1), (외무부, NNG, 2, 3), (對, SH, 6, 1...","[외무부, 쿠르드, 군사, 작전, 항의, 터키, 대사, 초치, 종합]"


In [221]:
world_cnt = list(world.loc[:, 'text_pos_filter'])
world_cnt = [data for inner_list in world_cnt for data in inner_list]
world_cnt = word_count(world_cnt)
world_cnt[:30]

[('종합', 121),
 ('트럼프', 94),
 ('이란', 80),
 ('터키', 44),
 ('미국', 42),
 ('시리아', 40),
 ('공격', 36),
 ('정부', 35),
 ('중국', 35),
 ('사망', 33),
 ('시위', 29),
 ('총리', 29),
 ('홍콩', 28),
 ('테러', 27),
 ('이스라엘', 26),
 ('대선', 26),
 ('사우디', 25),
 ('대통령', 24),
 ('제재', 23),
 ('일본', 22),
 ('하다', 21),
 ('협상', 21),
 ('합의', 20),
 ('힐러리', 20),
 ('러시아', 19),
 ('없다', 18),
 ('이라크', 18),
 ('오바마', 18),
 ('난민', 18),
 ('추진', 18)]

In [222]:
len(world_cnt)

3314

In [223]:
world_df = pd.DataFrame(world_cnt,columns=['word','cnt'])
world_df

,word,cnt
0,종합,121
1,트럼프,94
2,이란,80
3,터키,44
4,미국,42
...,...,...
3309,양회,1
3310,초치,1
3311,포르도,1
3312,기체,1


In [224]:
world_drop_df = world_df[world_df['cnt']>9]
world_drop_df

,word,cnt
0,종합,121
1,트럼프,94
2,이란,80
3,터키,44
4,미국,42
...,...,...
89,크다,10
90,사고,10
91,푸틴,10
92,시진핑,10


In [225]:
world_df.to_csv('world_df.csv',encoding='utf-8-sig')

In [226]:
world_drop_df.to_csv('world_drop_df.csv',encoding='utf-8-sig')

### 5) sport

In [227]:
sport['text_pos_tagging'] = sport['text'].progress_apply(lambda x: pos_tagging(x))

100%|██████████| 1000/1000 [00:01<00:00, 816.51it/s]
<ipython-input-227-d32ea796387e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sport['text_pos_tagging'] = sport['text'].progress_apply(lambda x: pos_tagging(x))


In [228]:
sport

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
5,ynat-v1_train_00005,배구연맹 이영일구 순천 KOVO커 부녕 대행업체 입찰 공고,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.07.10 16:47,"[(배구, NNG, 0, 2), (연맹, NNG, 2, 2), (이영일, NNP, ...","[배구, 연맹, 이영일, 순천, 크다, 부녕, 대행업체, 입찰, 공고]"
15,ynat-v1_train_00015,부상 고려하지 않은 ZiPS 오타니 타자로 20홈런·투...,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.02.06 09:50,"[(부상, NNG, 0, 2), (고려, NNG, 3, 2), (하, XSV, 5,...","[부상, 고려, 오타, 타자, 홈런]"
20,ynat-v1_train_00020,당대 최고 포수 양이지 올 시즌 성적 쩌도 신기함니다,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.06.13 22:37,"[(당대, NNG, 0, 2), (최고, NNG, 3, 2), (포수, NNG, 6...","[당대, 최고, 포수, 시즌, 성적, 신기하다]"
21,ynat-v1_train_00021,월드컵 주장 기성용은 왜 거짓말쟁이라는 단어를 썼나,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.06.08 06:05,"[(월드컵, NNG, 0, 3), (주장, NNG, 4, 2), (기성용, NNP,...","[월드컵, 주장, 기성용, 거짓말, 단어, 쓰다]"
22,ynat-v1_train_00022,아시안게임 응원석엔 하나된 함성 벤치엔 우정의 미소…...,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.08.15 23:34,"[(아시안, NNP, 0, 3), (게임, NNG, 3, 2), (응원석, NNG,...","[아시안, 게임, 응원석, 되다, 함성, 벤치, 우정, 미소]"
...,...,...,...,...,...,...,...
6971,ynat-v1_train_06971,로하스 kt 최초의 사이클링히트…삼성에 10점차 대승,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.05.29 22:41,"[(로하스, NNP, 0, 3), (kt, SL, 4, 2), (최초, NNG, 7...","[로하스, 최초, 사이클링, 히트, 삼성, 대승]"
6985,ynat-v1_train_06985,바르사알라베스 국왕컵 결승 레알 마드리드 안방에선 안돼,5,https://sports.news.naver.com/news.nhn?oid=001...,2017.02.14 09:24,"[(바르사알라베스, NNP, 0, 7), (국왕, NNG, 8, 2), (컵, NN...","[바르사알라베스, 국왕, 결승, 레알, 마드리드, 안방, 되다]"
6986,ynat-v1_train_06986,흥국생명 2위로도약…김희진 1점 IBK기업은행 5연패...,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.11.10 17:28,"[(흥국생명, NNP, 0, 4), (2, SN, 5, 1), (위, NNB, 6,...","[흥국생명, 도약, 김희, IBK기업은행, 연패]"
6994,ynat-v1_train_06994,월드컵 유니폼 입어본 선수들 가볍고 편하다,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.03.23 05:47,"[(월드컵, NNG, 0, 3), (유니폼, NNG, 4, 3), (입, VV-R,...","[월드컵, 유니폼, 선수, 편하다]"


In [229]:
sport['text_pos_filter'] = pos_filter(sport,'text_pos_tagging')

1000it [00:00, 9353.00it/s]
<ipython-input-229-b4bc8a70c9c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sport['text_pos_filter'] = pos_filter(sport,'text_pos_tagging')


In [230]:
sport

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
5,ynat-v1_train_00005,배구연맹 이영일구 순천 KOVO커 부녕 대행업체 입찰 공고,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.07.10 16:47,"[(배구, NNG, 0, 2), (연맹, NNG, 2, 2), (이영일, NNP, ...","[배구, 연맹, 이영일, 순천, 크다, 부녕, 대행업체, 입찰, 공고]"
15,ynat-v1_train_00015,부상 고려하지 않은 ZiPS 오타니 타자로 20홈런·투...,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.02.06 09:50,"[(부상, NNG, 0, 2), (고려, NNG, 3, 2), (하, XSV, 5,...","[부상, 고려, 오타, 타자, 홈런]"
20,ynat-v1_train_00020,당대 최고 포수 양이지 올 시즌 성적 쩌도 신기함니다,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.06.13 22:37,"[(당대, NNG, 0, 2), (최고, NNG, 3, 2), (포수, NNG, 6...","[당대, 최고, 포수, 시즌, 성적, 신기하다]"
21,ynat-v1_train_00021,월드컵 주장 기성용은 왜 거짓말쟁이라는 단어를 썼나,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.06.08 06:05,"[(월드컵, NNG, 0, 3), (주장, NNG, 4, 2), (기성용, NNP,...","[월드컵, 주장, 기성용, 거짓말, 단어, 쓰다]"
22,ynat-v1_train_00022,아시안게임 응원석엔 하나된 함성 벤치엔 우정의 미소…...,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.08.15 23:34,"[(아시안, NNP, 0, 3), (게임, NNG, 3, 2), (응원석, NNG,...","[아시안, 게임, 응원석, 되다, 함성, 벤치, 우정, 미소]"
...,...,...,...,...,...,...,...
6971,ynat-v1_train_06971,로하스 kt 최초의 사이클링히트…삼성에 10점차 대승,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.05.29 22:41,"[(로하스, NNP, 0, 3), (kt, SL, 4, 2), (최초, NNG, 7...","[로하스, 최초, 사이클링, 히트, 삼성, 대승]"
6985,ynat-v1_train_06985,바르사알라베스 국왕컵 결승 레알 마드리드 안방에선 안돼,5,https://sports.news.naver.com/news.nhn?oid=001...,2017.02.14 09:24,"[(바르사알라베스, NNP, 0, 7), (국왕, NNG, 8, 2), (컵, NN...","[바르사알라베스, 국왕, 결승, 레알, 마드리드, 안방, 되다]"
6986,ynat-v1_train_06986,흥국생명 2위로도약…김희진 1점 IBK기업은행 5연패...,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.11.10 17:28,"[(흥국생명, NNP, 0, 4), (2, SN, 5, 1), (위, NNB, 6,...","[흥국생명, 도약, 김희, IBK기업은행, 연패]"
6994,ynat-v1_train_06994,월드컵 유니폼 입어본 선수들 가볍고 편하다,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.03.23 05:47,"[(월드컵, NNG, 0, 3), (유니폼, NNG, 4, 3), (입, VV-R,...","[월드컵, 유니폼, 선수, 편하다]"


In [231]:
sport_cnt = list(sport.loc[:, 'text_pos_filter'])
sport_cnt = [data for inner_list in sport_cnt for data in inner_list]
sport_cnt = word_count(sport_cnt)
sport_cnt[:30]

[('감독', 97),
 ('농구', 81),
 ('프로', 71),
 ('경기', 69),
 ('배구', 66),
 ('월드컵', 61),
 ('여자', 57),
 ('축구', 56),
 ('선수', 55),
 ('시즌', 54),
 ('종합', 54),
 ('대표', 50),
 ('연승', 48),
 ('류현진', 47),
 ('리그', 42),
 ('홈런', 39),
 ('손흥민', 38),
 ('한국', 35),
 ('우승', 33),
 ('아시안', 28),
 ('꺾다', 28),
 ('출전', 28),
 ('연속', 28),
 ('연패', 27),
 ('승리', 27),
 ('게임', 26),
 ('야구', 25),
 ('두산', 25),
 ('추신수', 23),
 ('올림픽', 21)]

In [232]:
len(sport_cnt)

2640

In [233]:
sport_df = pd.DataFrame(sport_cnt,columns=['word','cnt'])
sport_df

,word,cnt
0,감독,97
1,농구,81
2,프로,71
3,경기,69
4,배구,66
...,...,...
2635,바르사알라베스,1
2636,김희,1
2637,편하다,1
2638,고통,1


In [234]:
sport_drop_df = sport_df[sport_df['cnt']>9]
sport_drop_df

,word,cnt
0,감독,97
1,농구,81
2,프로,71
3,경기,69
4,배구,66
...,...,...
80,탈출,10
81,상대,10
82,최초,10
83,대타,10


In [235]:
sport_df.to_csv('sport_df.csv',encoding='utf-8-sig')

In [236]:
sport_drop_df.to_csv('sport_drop_df.csv',encoding='utf-8-sig')

### 6) politics

In [237]:
politics['text_pos_tagging'] = politics['text'].progress_apply(lambda x: pos_tagging(x))

100%|██████████| 1000/1000 [00:01<00:00, 620.69it/s]
<ipython-input-237-53cd2ac22eac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  politics['text_pos_tagging'] = politics['text'].progress_apply(lambda x: pos_tagging(x))


In [238]:
politics

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
9,ynat-v1_train_00009,긴급비상대책위 참석하는 김병준과 나경원,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.02.08. 오후 6:32,"[(긴급, NNG, 0, 2), (비상, NNG, 2, 2), (대책위, NNG, ...","[긴급, 비상, 대책위, 참석, 김병준, 나경원]"
14,ynat-v1_train_00014,北 한미일 연합훈련 비난…지배주의적 야망의 발로,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.10.22. 오후 10:16,"[(北, SH, 0, 1), (한미일, NNP, 2, 3), (연합, NNG, 6,...","[한미일, 연합, 훈련, 비난, 지배, 주의, 야망]"
32,ynat-v1_train_00032,황총리 안보위기 평화통일 전기되도록 할 것,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.04.13. 오전 10:06,"[(황, NNP, 0, 1), (총리, NNG, 1, 2), (안보, NNG, 4,...","[총리, 안보, 위기, 평화, 통일, 전기]"
33,ynat-v1_train_00033,한국 계파갈등 재연 조짐…연이틀 비대위·친박 신경전,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.11.01. 오후 3:07,"[(한국, NNP, 0, 2), (계파, NNG, 3, 2), (갈등, NNG, 5...","[한국, 계파, 갈등, 재연, 조짐, 연이틀, 비대위, 신경전]"
39,ynat-v1_train_00039,불법·허위사실 vs 적법…곳곳서 여론조사 논란,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.02.09. 오전 8:00,"[(불법, NNG, 0, 2), (·, SP, 2, 1), (허위, NNG, 3, ...","[불법, 허위, 사실, 적법, 곳곳, 여론, 조사, 논란]"
...,...,...,...,...,...,...,...
6990,ynat-v1_train_06990,바른정당 창당 일련…궁미네당과 통하 푸에 녁싸소그로,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.21. 오전 8:00,"[(바른정당, NNP, 0, 4), (창당, NNG, 5, 2), (일련, NNG,...","[바른정당, 창당, 일련, …궁미네, 통하다, 푸에, 녁싸소그]"
6993,ynat-v1_train_06993,새해 첫 NSC 상임위 개최…北제안 후속조치 마련 착수,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.02. 오후 2:14,"[(새해, NNG, 0, 2), (첫, MM, 3, 1), (NSC, SL, 5, ...","[새해, 상임위, 개최, 제안, 후속, 조치, 마련, 착수]"
6996,ynat-v1_train_06996,정의장 사드 국회동의 사안 아니라 쳐도 충분히 협의해야,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.07.17. 오후 6:20,"[(정, NNP, 0, 1), (의장, NNG, 1, 2), (사드, NNG, 4,...","[의장, 사드, 국회, 동의, 사안, 치다, 협의]"
6997,ynat-v1_train_06997,정치권 엘시티 수사 돌발변수에 촉각…왜 지금,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.16. 오후 6:24,"[(정치, NNG, 0, 2), (권, XSN, 2, 1), (엘시티, NNP, 4...","[정치, 엘시티, 수사, 돌발, 변수, 촉각]"


In [239]:
politics['text_pos_filter'] = pos_filter(politics,'text_pos_tagging')

1000it [00:00, 9043.66it/s]
<ipython-input-239-5df955c932b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  politics['text_pos_filter'] = pos_filter(politics,'text_pos_tagging')


In [240]:
politics

,ID,text,target,url,date,text_pos_tagging,text_pos_filter
9,ynat-v1_train_00009,긴급비상대책위 참석하는 김병준과 나경원,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.02.08. 오후 6:32,"[(긴급, NNG, 0, 2), (비상, NNG, 2, 2), (대책위, NNG, ...","[긴급, 비상, 대책위, 참석, 김병준, 나경원]"
14,ynat-v1_train_00014,北 한미일 연합훈련 비난…지배주의적 야망의 발로,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.10.22. 오후 10:16,"[(北, SH, 0, 1), (한미일, NNP, 2, 3), (연합, NNG, 6,...","[한미일, 연합, 훈련, 비난, 지배, 주의, 야망]"
32,ynat-v1_train_00032,황총리 안보위기 평화통일 전기되도록 할 것,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.04.13. 오전 10:06,"[(황, NNP, 0, 1), (총리, NNG, 1, 2), (안보, NNG, 4,...","[총리, 안보, 위기, 평화, 통일, 전기]"
33,ynat-v1_train_00033,한국 계파갈등 재연 조짐…연이틀 비대위·친박 신경전,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.11.01. 오후 3:07,"[(한국, NNP, 0, 2), (계파, NNG, 3, 2), (갈등, NNG, 5...","[한국, 계파, 갈등, 재연, 조짐, 연이틀, 비대위, 신경전]"
39,ynat-v1_train_00039,불법·허위사실 vs 적법…곳곳서 여론조사 논란,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.02.09. 오전 8:00,"[(불법, NNG, 0, 2), (·, SP, 2, 1), (허위, NNG, 3, ...","[불법, 허위, 사실, 적법, 곳곳, 여론, 조사, 논란]"
...,...,...,...,...,...,...,...
6990,ynat-v1_train_06990,바른정당 창당 일련…궁미네당과 통하 푸에 녁싸소그로,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.21. 오전 8:00,"[(바른정당, NNP, 0, 4), (창당, NNG, 5, 2), (일련, NNG,...","[바른정당, 창당, 일련, …궁미네, 통하다, 푸에, 녁싸소그]"
6993,ynat-v1_train_06993,새해 첫 NSC 상임위 개최…北제안 후속조치 마련 착수,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.02. 오후 2:14,"[(새해, NNG, 0, 2), (첫, MM, 3, 1), (NSC, SL, 5, ...","[새해, 상임위, 개최, 제안, 후속, 조치, 마련, 착수]"
6996,ynat-v1_train_06996,정의장 사드 국회동의 사안 아니라 쳐도 충분히 협의해야,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.07.17. 오후 6:20,"[(정, NNP, 0, 1), (의장, NNG, 1, 2), (사드, NNG, 4,...","[의장, 사드, 국회, 동의, 사안, 치다, 협의]"
6997,ynat-v1_train_06997,정치권 엘시티 수사 돌발변수에 촉각…왜 지금,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.16. 오후 6:24,"[(정치, NNG, 0, 2), (권, XSN, 2, 1), (엘시티, NNP, 4...","[정치, 엘시티, 수사, 돌발, 변수, 촉각]"


In [241]:
politics_cnt = list(politics.loc[:, 'text_pos_filter'])
politics_cnt = [data for inner_list in politics_cnt for data in inner_list]
politics_cnt = word_count(politics_cnt)
politics_cnt[:30]

[('대통령', 187),
 ('종합', 145),
 ('회담', 65),
 ('정상', 63),
 ('국회', 56),
 ('김정은', 38),
 ('정부', 36),
 ('대표', 34),
 ('남북', 31),
 ('여야', 30),
 ('하다', 30),
 ('평양', 29),
 ('회의', 29),
 ('의원', 27),
 ('북한', 26),
 ('선거', 26),
 ('한국당', 25),
 ('없다', 25),
 ('가능', 25),
 ('총리', 24),
 ('장관', 24),
 ('선언', 23),
 ('민주', 23),
 ('방문', 22),
 ('국민', 22),
 ('대회', 21),
 ('대화', 21),
 ('평화', 20),
 ('더민주', 18),
 ('대북', 18)]

In [242]:
len(politics_cnt)

2977

In [243]:
politics_df = pd.DataFrame(politics_cnt,columns=['word','cnt'])
politics_df

,word,cnt
0,대통령,187
1,종합,145
2,회담,65
3,정상,63
4,국회,56
...,...,...
2972,녁싸소그,1
2973,후속,1
2974,엘시티,1
2975,객석,1


In [244]:
politics_drop_df = politics_df[politics_df['cnt']>9]
politics_drop_df

,word,cnt
0,대통령,187
1,종합,145
2,회담,65
3,정상,63
4,국회,56
...,...,...
86,개헌,10
87,김종인,10
88,있다,10
89,채택,10


In [245]:
politics_df.to_csv('politics_df.csv',encoding='utf-8-sig')

In [259]:
politics_drop_df.to_csv('politics_drop_df.csv',encoding='utf-8-sig')

## 합쳐서 비율 확인(**_drop_df)

### 열 이름 변경

In [251]:
it_science_drop_df = it_science_drop_df.rename(columns={'cnt':'it_science_cnt'})

In [252]:
it_science_drop_df

,word,it_science_cnt
0,출시,100
1,개발,80
2,기술,74
3,종합,69
4,삼성,47
...,...,...
84,공동,10
85,확인,10
86,알파고,10
87,활용,10


In [253]:
economy_drop_df = economy_drop_df.rename(columns={'cnt':'economy_cnt'})
economy_drop_df

,word,economy_cnt
0,종합,92
1,분기,91
2,투자,71
3,작년,48
4,실적,44
...,...,...
101,개월,10
102,추가,10
103,개선,10
104,업체,10


In [254]:
society_drop_df = society_drop_df.rename(columns={'cnt':'society_cnt'})
society_drop_df

,word,society_cnt
0,종합,77
1,게시판,43
2,코로나,40
3,지원,29
4,한국,24
...,...,...
78,등교,10
79,정책,10
80,시민,10
81,간담회,10


In [255]:
culture_drop_df = culture_drop_df.rename(columns={'cnt':'culture_cnt'})
culture_drop_df

,word,culture_cnt
0,신간,103
1,여행,48
2,축제,44
3,서울,39
4,공연,38
...,...,...
68,남부,10
69,서다,10
70,예술,10
71,국제,10


In [256]:
world_drop_df = world_drop_df.rename(columns={'cnt':'world_cnt'})
world_drop_df

,word,world_cnt
0,종합,121
1,트럼프,94
2,이란,80
3,터키,44
4,미국,42
...,...,...
89,크다,10
90,사고,10
91,푸틴,10
92,시진핑,10


In [257]:
sport_drop_df = sport_drop_df.rename(columns={'cnt':'sport_cnt'})
sport_drop_df

,word,sport_cnt
0,감독,97
1,농구,81
2,프로,71
3,경기,69
4,배구,66
...,...,...
80,탈출,10
81,상대,10
82,최초,10
83,대타,10


In [260]:
politics_drop_df = politics_drop_df.rename(columns={'cnt':'politics_cnt'})
politics_drop_df

,word,politics_cnt
0,대통령,187
1,종합,145
2,회담,65
3,정상,63
4,국회,56
...,...,...
86,개헌,10
87,김종인,10
88,있다,10
89,채택,10


### 데이터 프레임 merge

In [263]:
from functools import reduce

result = reduce(lambda x,y: pd.merge(x,y, on='word', how='outer'), [it_science_drop_df,
                                                                   economy_drop_df,
                                                                   society_drop_df,
                                                                   culture_drop_df,
                                                                   world_drop_df,
                                                                   sport_drop_df,
                                                                   politics_drop_df])
result

,word,it_science_cnt,economy_cnt,society_cnt,culture_cnt,world_cnt,sport_cnt,politics_cnt
0,출시,100.0,37.0,NaN,NaN,NaN,NaN,NaN
1,개발,80.0,NaN,11.0,NaN,NaN,NaN,NaN
2,기술,74.0,NaN,NaN,NaN,NaN,NaN,NaN
3,종합,69.0,92.0,77.0,21.0,121.0,54.0,145.0
4,삼성,47.0,23.0,NaN,NaN,NaN,14.0,NaN
...,...,...,...,...,...,...,...,...
489,결과,NaN,NaN,NaN,NaN,NaN,NaN,10.0
490,입장,NaN,NaN,NaN,NaN,NaN,NaN,10.0
491,개헌,NaN,NaN,NaN,NaN,NaN,NaN,10.0
492,김종인,NaN,NaN,NaN,NaN,NaN,NaN,10.0


### 열 합계

In [266]:
result['total_cnt']=result.sum(axis=1)
result

<ipython-input-266-3d5a7d881cd9>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  result['total_cnt']=result.sum(axis=1)


,word,it_science_cnt,economy_cnt,society_cnt,culture_cnt,world_cnt,sport_cnt,politics_cnt,total_cnt
0,출시,100.0,37.0,NaN,NaN,NaN,NaN,NaN,137.0
1,개발,80.0,NaN,11.0,NaN,NaN,NaN,NaN,91.0
2,기술,74.0,NaN,NaN,NaN,NaN,NaN,NaN,74.0
3,종합,69.0,92.0,77.0,21.0,121.0,54.0,145.0,579.0
4,삼성,47.0,23.0,NaN,NaN,NaN,14.0,NaN,84.0
...,...,...,...,...,...,...,...,...,...
489,결과,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0
490,입장,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0
491,개헌,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0
492,김종인,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0


### 각 카테고리 별 단어 비율

In [268]:
result = result.fillna(0)
result

,word,it_science_cnt,economy_cnt,society_cnt,culture_cnt,world_cnt,sport_cnt,politics_cnt,total_cnt
0,출시,100.0,37.0,0.0,0.0,0.0,0.0,0.0,137.0
1,개발,80.0,0.0,11.0,0.0,0.0,0.0,0.0,91.0
2,기술,74.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0
3,종합,69.0,92.0,77.0,21.0,121.0,54.0,145.0,579.0
4,삼성,47.0,23.0,0.0,0.0,0.0,14.0,0.0,84.0
...,...,...,...,...,...,...,...,...,...
489,결과,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0
490,입장,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0
491,개헌,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0
492,김종인,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0


In [289]:
def per_percent(df,col1):
  result = []
  for index, row in tqdm(df.iterrows()):
    if row[col1]!=0:
      result.append(round(row[col1]/row['total_cnt'],3)*100)
    else:
      result.append(0)
  return result

In [290]:
result['it_science_percent'] = per_percent(result,'it_science_cnt')

494it [00:00, 17972.57it/s]


In [292]:
result['economy_percent'] = per_percent(result,'economy_cnt')

494it [00:00, 5750.72it/s]


In [293]:
result['society_percent'] = per_percent(result,'society_cnt')

494it [00:00, 15496.00it/s]


In [294]:
result['culture_percent'] = per_percent(result,'culture_cnt')

494it [00:00, 16161.89it/s]


In [295]:
result['world_percent'] = per_percent(result,'world_cnt')

494it [00:00, 3403.95it/s]


In [296]:
result['sport_percent'] = per_percent(result,'sport_cnt')

494it [00:00, 13002.82it/s]


In [297]:
result['politics_percent'] = per_percent(result,'politics_cnt')

494it [00:00, 8951.11it/s]


In [298]:
result

,word,it_science_cnt,economy_cnt,society_cnt,culture_cnt,world_cnt,sport_cnt,politics_cnt,total_cnt,it_science_percent,economy_percent,society_percent,culture_percent,world_percent,sport_percent,politics_percent
0,출시,100.0,37.0,0.0,0.0,0.0,0.0,0.0,137.0,73.0,27.0,0.0,0.0,0.0,0.0,0.0
1,개발,80.0,0.0,11.0,0.0,0.0,0.0,0.0,91.0,87.9,0.0,12.1,0.0,0.0,0.0,0.0
2,기술,74.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
3,종합,69.0,92.0,77.0,21.0,121.0,54.0,145.0,579.0,11.9,15.9,13.3,3.6,20.9,9.3,25.0
4,삼성,47.0,23.0,0.0,0.0,0.0,14.0,0.0,84.0,56.0,27.4,0.0,0.0,0.0,16.7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,결과,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
490,입장,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
491,개헌,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
492,김종인,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0


In [330]:
result.to_csv('category_per_result.csv',encoding='utf-8-sig')

### 데이터 프레임 분리 후 저장

In [303]:
it_science_result = result[['word','it_science_cnt','it_science_percent']]

In [308]:
it_science_result

,word,it_science_cnt,it_science_percent
0,출시,100.0,73.0
1,개발,80.0,87.9
2,기술,74.0,100.0
3,종합,69.0,11.9
4,삼성,47.0,56.0
...,...,...,...
489,결과,0.0,0.0
490,입장,0.0,0.0
491,개헌,0.0,0.0
492,김종인,0.0,0.0


In [309]:
it_science_result.to_csv('it_science_result.csv',encoding='utf-8-sig')

In [310]:
economy_result = result[['word','economy_cnt','economy_percent']]

In [311]:
economy_result

,word,economy_cnt,economy_percent
0,출시,37.0,27.0
1,개발,0.0,0.0
2,기술,0.0,0.0
3,종합,92.0,15.9
4,삼성,23.0,27.4
...,...,...,...
489,결과,0.0,0.0
490,입장,0.0,0.0
491,개헌,0.0,0.0
492,김종인,0.0,0.0


In [312]:
economy_result.to_csv('economy_result.csv',encoding='utf-8-sig')

In [313]:
society_result = result[['word','society_cnt','society_percent']]

In [315]:
society_result

,word,society_cnt,society_percent
0,출시,0.0,0.0
1,개발,11.0,12.1
2,기술,0.0,0.0
3,종합,77.0,13.3
4,삼성,0.0,0.0
...,...,...,...
489,결과,0.0,0.0
490,입장,0.0,0.0
491,개헌,0.0,0.0
492,김종인,0.0,0.0


In [316]:
society_result.to_csv('society_result.csv',encoding='utf-8-sig')

In [317]:
culture_result = result[['word','culture_cnt','culture_percent']]

In [319]:
culture_result

,word,culture_cnt,culture_percent
0,출시,0.0,0.0
1,개발,0.0,0.0
2,기술,0.0,0.0
3,종합,21.0,3.6
4,삼성,0.0,0.0
...,...,...,...
489,결과,0.0,0.0
490,입장,0.0,0.0
491,개헌,0.0,0.0
492,김종인,0.0,0.0


In [321]:
culture_result.to_csv('culture_result.csv',encoding='utf-8-sig')

In [322]:
world_result = result[['word','world_cnt','world_percent']]

In [323]:
world_result

,word,world_cnt,world_percent
0,출시,0.0,0.0
1,개발,0.0,0.0
2,기술,0.0,0.0
3,종합,121.0,20.9
4,삼성,0.0,0.0
...,...,...,...
489,결과,0.0,0.0
490,입장,0.0,0.0
491,개헌,0.0,0.0
492,김종인,0.0,0.0


In [324]:
world_result.to_csv('world_result.csv',encoding='utf-8-sig')

In [325]:
sport_result = result[['word','sport_cnt','sport_percent']]

In [326]:
sport_result

,word,sport_cnt,sport_percent
0,출시,0.0,0.0
1,개발,0.0,0.0
2,기술,0.0,0.0
3,종합,54.0,9.3
4,삼성,14.0,16.7
...,...,...,...
489,결과,0.0,0.0
490,입장,0.0,0.0
491,개헌,0.0,0.0
492,김종인,0.0,0.0


In [327]:
sport_result.to_csv('sport_result.csv',encoding='utf-8-sig')

In [328]:
politics_result = result[['word','politics_cnt','politics_percent']]

In [329]:
politics_result.to_csv('politics_result.csv',encoding='utf-8-sig')